vgg 16
resnet 18, 50

In [1]:
import sys
import os

# Add parent directory to Python path
current_dir = os.path.abspath('.')
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from torchvision import transforms
from torch.utils import data
import numpy as np
import argparse
import torch
from torchvision.models import resnet50, ResNet50_Weights

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(42)


In [2]:
DATASET_ROOT = '../CUB/DATASET/CUB_200_2011'  # Updated path to go up one directory
DATASET_VALIDATION_RANDOM_SEED = 123
BATCH_SIZE = 8
DATASET_WORK_NUMBER = 8
DATASET_SPLIT_RATIO = 0.8

In [3]:
from DatasetLoader.cub import CUB

trans_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees=30, translate=(0.1,0.1), scale=(0.8,1.2)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
    transforms.RandomResizedCrop(224,scale=(0.7,1.0), ratio=(3/4, 4/3)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

trans_test = transforms.Compose([
    # ATTENZIONE QUA NON DOVREBBE SERIVIRE IL RESIZE (?)
    transforms.Resize((224, 224)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# create dataset
train_data = CUB(DATASET_ROOT, 'train', DATASET_SPLIT_RATIO, DATASET_VALIDATION_RANDOM_SEED, transform=trans_train)
valid_data = CUB(DATASET_ROOT, 'valid', DATASET_SPLIT_RATIO, DATASET_VALIDATION_RANDOM_SEED, transform=trans_test)
test_data = CUB(DATASET_ROOT, 'test', 0, 0, transform=trans_test)

print("Train: {}".format(len(train_data)))
print("Valid: {}".format(len(valid_data)))
print("Test: {}".format(len(test_data)))

Train: 4796
Valid: 1198
Test: 5794


In [4]:
# create dataloader
train_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, num_workers=DATASET_WORK_NUMBER)
valid_loader = data.DataLoader(valid_data, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=DATASET_WORK_NUMBER)
test_loader = data.DataLoader(test_data, batch_size=BATCH_SIZE*2, shuffle=False, num_workers=DATASET_WORK_NUMBER)

In [5]:
model = resnet50(ResNet50_Weights.IMAGENET1K_V2)
model.fc = torch.nn.Linear(in_features=2048, out_features=200, bias=True)

#freeze parameters other than fc
#for param in model.parameters():
#    param.requires_grad = False
#for param in model.fc.parameters():
#    param.requires_grad = True
print(model.fc)

model.to(device)

/home/matteo/miniforge3/envs/mambaLRPTEST/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


Linear(in_features=2048, out_features=200, bias=True)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.val_loss_min = val_loss
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.val_loss_min = val_loss
            self.counter = 0

In [7]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

loss_function = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)

early_stopping = EarlyStopping(patience=10, verbose=True, delta=0.05)
num_epochs = 1000
train_losses, val_losses = [], []

In [8]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    train_correct = 0
    train_total = 0

    for images, labels, _ in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = outputs.max(1)
        train_correct += predicted.eq(labels).sum().item()
        train_total += labels.size(0)

        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)
    train_acc = 100 * train_correct / train_total
    # Validation phase
    model.eval()
    val_running_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels, _ in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_function(outputs, labels)
            val_running_loss += loss.item()
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()
            val_total += labels.size(0)
    val_loss = val_running_loss / len(valid_loader)
    val_losses.append(val_loss)
    val_acc = 100 * val_correct / val_total

    # Print the losses for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] → Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

    # Step the learning rate scheduler
    scheduler.step(val_loss)

    
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break
    

Epoch [1/1000] → Train Loss: 5.3081 | Val Loss: 5.2619, Train Acc: 0.50% | Val Acc: 0.42%


/home/matteo/miniforge3/envs/mambaLRPTEST/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:204: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch [2/1000] → Train Loss: 5.2056 | Val Loss: 8.2828, Train Acc: 1.02% | Val Acc: 0.33%
EarlyStopping counter: 1 out of 10
Epoch [3/1000] → Train Loss: 5.0700 | Val Loss: 5.0310, Train Acc: 1.02% | Val Acc: 1.34%
Epoch [4/1000] → Train Loss: 5.0422 | Val Loss: 5.1255, Train Acc: 1.27% | Val Acc: 1.17%
EarlyStopping counter: 1 out of 10
Epoch [5/1000] → Train Loss: 4.9607 | Val Loss: 5.0643, Train Acc: 1.36% | Val Acc: 1.50%
EarlyStopping counter: 2 out of 10
Epoch [6/1000] → Train Loss: 4.9075 | Val Loss: 4.9413, Train Acc: 1.42% | Val Acc: 2.17%
Epoch [7/1000] → Train Loss: 4.8584 | Val Loss: 4.7422, Train Acc: 2.02% | Val Acc: 2.75%
Epoch [8/1000] → Train Loss: 4.7496 | Val Loss: 4.6678, Train Acc: 2.96% | Val Acc: 4.34%
Epoch [9/1000] → Train Loss: 4.5753 | Val Loss: 4.5682, Train Acc: 4.32% | Val Acc: 5.26%
Epoch [10/1000] → Train Loss: 4.4284 | Val Loss: 4.3271, Train Acc: 5.40% | Val Acc: 5.84%
Epoch [11/1000] → Train Loss: 4.2994 | Val Loss: 4.1747, Train Acc: 5.94% | Val Acc:

In [ ]:
checkpoint_path = 'resnet50_cub_finetuned_all_weights_DU.pth'

In [10]:
torch.save(model.state_dict(), checkpoint_path)

In [11]:
checkpoint_fine_tuned = torch.load(checkpoint_path)
status = model.load_state_dict(checkpoint_fine_tuned, strict=False)

In [12]:
print(device)
model.to(device)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels, _ in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _ , predicted = torch.max(outputs, 1)  # ottiene la classe con punteggio massimo

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuratezza finale: {accuracy:.2f}%')

cuda
Accuratezza finale: 43.94%
